In [ ]:
import numpy as np
import random

# Lévy flight function
def levy_flight(Lambda, dim):
    # Step size for Lévy flights
    u = np.random.normal(0, 1, dim)  # Generate normal random values for u
    v = np.random.normal(0, 1, dim)  # Generate normal random values for v
    step = u / (np.power(np.abs(v), 1 / Lambda))  # Lévy flight formula
    return step

# Objective function (for testing: minimize this function)
def objective_function(x):
    return np.sum(x ** 2)

# Cuckoo Search Algorithm
def cuckoo_search(obj_func, n_nests, max_iter, dim, bounds=(-5, 5), Pa=0.25, alpha=0.01, Lambda=1.5):
    # Initialize nests randomly within the bounds
    nests = np.random.uniform(bounds[0], bounds[1], (n_nests, dim))
    fitness = np.apply_along_axis(obj_func, 1, nests)
    # Best solution initialization
    best_idx = np.argmin(fitness)
    best_solution = nests[best_idx]
    best_fitness = fitness[best_idx]
    # Main loop (iterations)
    for t in range(max_iter):
        # Generate new solutions via Lévy flights
        for i in range(n_nests):
            step = alpha * levy_flight(Lambda, dim)  # Generate Lévy flight
            new_solution = nests[i] + step
            # Ensure the new solution stays within bounds
            new_solution = np.clip(new_solution, bounds[0], bounds[1])
            new_fitness = obj_func(new_solution)
            # If the new solution is better, replace the old one
            if new_fitness < fitness[i]:
                nests[i] = new_solution
                fitness[i] = new_fitness
        # Abandon the worst nests with probability Pa
        for i in range(n_nests):
            if random.random() < Pa:
                nests[i] = np.random.uniform(bounds[0], bounds[1], dim)  # Random solution
                fitness[i] = obj_func(nests[i])
        # Update the best solution
        current_best_idx = np.argmin(fitness)
        current_best_solution = nests[current_best_idx]
        current_best_fitness = fitness[current_best_idx]
        # Track the best solution
        if current_best_fitness < best_fitness:
            best_solution = current_best_solution
            best_fitness = current_best_fitness
    return best_solution, best_fitness

# Parameters
n_nests = 50           # Number of nests
max_iter = 100         # Maximum iterations
dim = 5                # Dimensionality of the problem
bounds = (-5, 5)       # Bounds for the solution space
Pa = 0.25              # Probability of abandoning a nest
alpha = 0.01           # Step size
Lambda = 1.5           # Lévy flight parameter

# Run the cuckoo search algorithm
best_solution, best_fitness = cuckoo_search(objective_function, n_nests, max_iter, dim, bounds, Pa, alpha, Lambda)

print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)


Best Solution: [ 2.22910894  2.79883343  0.22644073  2.08040217 -2.39297997]
Best Fitness: 2.7382787196957485


In [2]:
#ENGINEERING DESIGN APPLICATION

import numpy as np

# Constants
NUM_NESTS = 25  # Number of nests
NUM_ITERATIONS = 10  # Number of iterations
PA = 0.25  # Probability of abandoning a nest
L = 100  # Length of the truss (dummy value for simplicity)
RHO = 7.85e3  # Density of steel (kg/m^3) for the material

# Function to calculate the weight of the truss structure
def calculate_weight(areas, lengths, rho=RHO):
    return np.sum(areas * lengths * rho)

# Function to calculate the deflection (simplified model for demonstration purposes)
def calculate_deflection(areas, lengths):
    # Example: Deflection = Sum of inverse areas (simplified for the demonstration)
    return np.sum(1 / areas)

# Objective function: Minimize weight while satisfying deflection constraint
def objective_function(areas, lengths, deflection_limit=10):
    weight = calculate_weight(areas, lengths)
    deflection = calculate_deflection(areas, lengths)
    if deflection > deflection_limit:
        return np.inf  # Return an infeasible solution if deflection exceeds the limit
    return weight

# Generate a random initial solution (cross-sectional areas)
def initialize_nests(num_nests, num_beams):
    return np.random.uniform(low=0.1, high=10.0, size=(num_nests, num_beams))

# Cuckoo Search Algorithm
def cuckoo_search(num_nests, num_iterations, areas_range=(0.1, 10.0), lengths=None):
    num_beams = len(lengths) if lengths is not None else 5  # Default: 5 beams
    nests = initialize_nests(num_nests, num_beams)  # Initialize nests with random solutions
    fitness = np.array([objective_function(nest, lengths) for nest in nests])  # Evaluate initial solutions
    best_nest = nests[np.argmin(fitness)]  # Best nest with minimum weight
    best_fitness = np.min(fitness)

    for iteration in range(num_iterations):
        # Generate new solutions (cuckoo eggs)
        new_nests = nests + np.random.randn(num_nests, num_beams) * 0.1  # Levy flight step

        # Ensure the solutions are within the allowable range
        new_nests = np.clip(new_nests, areas_range[0], areas_range[1])

        # Evaluate the new nests
        new_fitness = np.array([objective_function(nest, lengths) for nest in new_nests])

        # Abandon nests if necessary (based on probability PA)
        abandon = np.random.rand(num_nests) < PA
        nests[abandon] = new_nests[abandon]
        fitness[abandon] = new_fitness[abandon]

        # Update the best nest
        min_idx = np.argmin(fitness)
        if fitness[min_idx] < best_fitness:
            best_fitness = fitness[min_idx]
            best_nest = nests[min_idx]

        # Print the progress
        print(f"Iteration {iteration+1}/{num_iterations}: Best Fitness = {best_fitness}")

    return best_nest, best_fitness

# Example usage:
lengths = [5, 5, 5, 5, 5]  # Length of each beam (in meters)
best_design, best_weight = cuckoo_search(NUM_NESTS, NUM_ITERATIONS, lengths=lengths)

print("Best Design (Cross-sectional areas):", best_design)
print("Best Weight:", best_weight)


Iteration 1/10: Best Fitness = 355357.8430367811
Iteration 2/10: Best Fitness = 355357.8430367811
Iteration 3/10: Best Fitness = 355357.8430367811
Iteration 4/10: Best Fitness = 355357.8430367811
Iteration 5/10: Best Fitness = 355357.8430367811
Iteration 6/10: Best Fitness = 352428.8428649454
Iteration 7/10: Best Fitness = 352428.8428649454
Iteration 8/10: Best Fitness = 352428.8428649454
Iteration 9/10: Best Fitness = 352428.8428649454
Iteration 10/10: Best Fitness = 352428.8428649454
Best Design (Cross-sectional areas): [0.30382941 1.76710058 0.80922782 0.70894598 5.5213804 ]
Best Weight: 352428.8428649454
